# Neural Machine Translation with RNN

We apply RNN to the language translation task, which is a typical seq to seq learning tastk. We will use a Encoder-Decoder architecture, where we use RNN models for both Encoder and Decoder parts. 

We will use the data from [the Tatoeba Project](http://www.manythings.org/anki/) and specifically, we will start with the fra-eng (French-English) translation data.

# Data Processing

In [49]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from random import shuffle
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

## Read raw texts and perform train-val-test splitting

In [31]:
## Read all lines and shuffle
file = open("datasets/fra-eng/fra.txt", "r")
all_lines = file.readlines()
shuffle(all_lines)
## Train-Val-Test split
train_ratio, val_ratio = 0.8, 0.1
train_end = int(len(all_lines)*train_ratio)
val_end = int(len(all_lines)*(train_ratio+val_ratio))
train_raw = all_lines[:train_end]
val_raw = all_lines[train_end:val_end]
test_raw = all_lines[val_end:]

## Vocab, Dataset and Dataloader

In [61]:
def get_vocabs(lines, source_tokenizer, target_tokenizer):
    source_counter = Counter()
    target_counter = Counter()
    for line in lines:
        parts = line.split("\t") #source, target and note are separated by tab
        source_counter.update(source_tokenizer(parts[0].lower()))
        target_counter.update(target_tokenizer(parts[1].lower()))
    source_vocab = vocab(source_counter, specials=["<unk>","<pad>", "<bos>", "<eos>"])
    target_vocab = vocab(target_counter, specials=["<unk>","<pad>", "<bos>", "<eos>"])
    source_vocab.set_default_index(source_vocab['<unk>'])
    target_vocab.set_default_index(target_vocab['<unk>'])    
    return source_vocab, target_vocab

def process_data(lines, source_tokenizer, target_tokenizer, source_vocab, target_vocab):
    data=[]
    for line in lines:
        source, target, _ = line.split("\t")
        source_tensor = torch.tensor(source_vocab(source_tokenizer(source.lower())))
        target_tensor = torch.tensor(target_vocab(target_tokenizer(target.lower())))
        data.append((source_tensor, target_tensor))
    return data

def generate_batch(data_batch, source_vocab, target_vocab):
    source_batch = []
    target_batch = []
    for source, target in data_batch:
        source_batch.append(torch.cat([torch.tensor(source_vocab["<bos>"]), source, torch.tensor([source_vocab["<eos>"]])], dim=0))
        target_batch.append(torch.cat([torch.tensor([target_vocab["<bos>"]]), target, torch.tensor([target_vocab["<eos>"]])], dim=0))
    source_batch = pad_sequence(source_batch, padding_value=source_vocab["<pad>"])
    target_batch = pad_sequence(target_batch, padding_value=target_vocab["<pad>"])
    return source_batch, target_batch

In [63]:
## Use tokenizers from spacy
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
fr_tokenizer = get_tokenizer("spacy", language="fr_core_news_sm")
## Get source target vocabs
en_vocab, fr_vocab = get_vocabs(train_raw, en_tokenizer, fr_tokenizer)
## Get source tartget token ids
train_data = process_data(train_raw, en_tokenizer, fr_tokenizer, en_vocab, fr_vocab)
val_data = process_data(val_raw, en_tokenizer, fr_tokenizer, en_vocab, fr_vocab)
test_data = process_data(test_raw, en_tokenizer, fr_tokenizer, en_vocab, fr_vocab)
## Get train, val, test dataloader
collator = lambda x: generate_batch(x, en_vocab, fr_vocab)
bsz = 16
train_iter = DataLoader(train_data, batch_size=bsz, shuffle=True, collate_fn=collator,num_workers=8)
val_iter = DataLoader(val_data, batch_size=bsz, shuffle=False, collate_fn=collator, num_workers=8)
test_iter = DataLoader(test_data, batch_size=bsz, shuffle=False, collate_fn=collator, num_workers=8)

# Building Seq2Seq Module

In [65]:
from torch import nn 
import torch.nn.functional as F 
import random

In [70]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(2*enc_hid_dim, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embeded = self.embedding(src)
        outputs, hidden = self.rnn(embeded)
        hidden = torch.tanh(self.fc(torch.cat([hidden[-2,:,:],hidden[-1,:,:]],dim=-1)))

        return outputs, hidden

class Attention(nn.Module):
    def __init__(self,enc_hid_dim, dec_hid_dim, attn_dim):
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim*2) + dec_hid_dim 
        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self, decoder_hidden, encoder_outputs):
        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        encdoer_outputs = encoder_outputs.permute(1,0,2)

        energy = torch.tanh(
            self.attn(torch.cat([repeated_decoder_hidden, encoder_outputs], dim=-1))
        )
        attention = torch.sum(energy, dim=-1)
        return F.softmax(attention, dim=-1)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(enc_hid_dim*2+emb_dim, dec_hid_dim)
        self.out = nn.Linear(self.attention.attn_in+emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def _weighted_encoder_rep(self, decoder_hidden, encoder_outputs):
        a = self.attention(decoder_hidden, encoder_outputs)
        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1,0,2)
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        weighted_encoder_rep = weighted_encoder_rep.permute(1,0,2)

        return weighted_encoder_rep
    
    def forward(self, input, decoder_hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embeded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden, encoder_outputs)

        rnn_input = torch.cat((embeded, weighted_encoder_rep),dim=2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embeded = embeded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output, weighted_encoder_rep, embeded),dim=1))
        return output, decoder_hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1] # tensor.max() returns tuple of maxiums and idmax
            output = (trg[t] if teacher_force else top1)
        return outputs

torch.return_types.max(
values=tensor([2, 5, 8]),
indices=tensor([2, 2, 2]))